<a href="https://colab.research.google.com/github/shivangini831-sys/FitPulse-Health-Anomaly-Detection-From-Fitness-Devices/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok plotly scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.1 MB/s eta 0:00:00


In [2]:
import pandas as pd

heart_df = pd.read_csv("heart_rate.csv")
heart_df.head()


,timestamp,heart_rate
0,2024-01-01 00:00:00,72.5
1,2024-01-01 00:15:00,69.7
2,2024-01-01 00:30:00,74.0
3,2024-01-01 00:45:00,78.8
4,2024-01-01 01:00:00,70.4


In [3]:
sleep_df = pd.read_csv("sleep.csv")
sleep_df.head()


,date,sleep_hours
0,2024-01-01,6.83
1,2024-01-02,6.55
2,2024-01-03,7.70
3,2024-01-04,7.96
4,2024-01-05,7.09


In [4]:
steps_df = pd.read_csv("steps.csv")
steps_df.head()


,timestamp,steps
0,2024-01-01 00:00:00,22
1,2024-01-01 00:15:00,21
2,2024-01-01 00:30:00,13
3,2024-01-01 00:45:00,16
4,2024-01-01 01:00:00,15


In [5]:
heart_df.rename(columns={"Date": "date", "HeartRate": "heart_rate"}, inplace=True)
sleep_df.rename(columns={"Sleep": "sleep_hours"}, inplace=True)
steps_df.rename(columns={"StepCount": "steps"}, inplace=True)


In [8]:
heart_df.rename(columns={
    "Date": "date",
    "HeartRate": "heart_rate"
}, inplace=True)


In [9]:
sleep_df.rename(columns={
    "SleepDate": "date",
    "TotalSleepHours": "sleep_hours"
}, inplace=True)


In [10]:
steps_df.rename(columns={
    "ActivityDay": "date",
    "StepTotal": "steps"
}, inplace=True)


In [11]:
['date', 'heart_rate']
['date', 'sleep_hours']
['date', 'steps']


['date', 'steps']

In [14]:
import pandas as pd

heart_df = pd.read_csv("heart_rate.csv")
print("Heart Rate Columns:", heart_df.columns.tolist())


Heart Rate Columns: ['timestamp', 'heart_rate']


In [15]:
import pandas as pd

heart_df = pd.read_csv("heart_rate.csv")

# Rename timestamp to date
heart_df.rename(columns={"timestamp": "date"}, inplace=True)

# Convert to datetime
heart_df['date'] = pd.to_datetime(heart_df['date'])

print(heart_df.columns)
heart_df.head()


Index(['date', 'heart_rate'], dtype='object')


,date,heart_rate
0,2024-01-01 00:00:00,72.5
1,2024-01-01 00:15:00,69.7
2,2024-01-01 00:30:00,74.0
3,2024-01-01 00:45:00,78.8
4,2024-01-01 01:00:00,70.4


In [16]:
sleep_df = pd.read_csv("sleep.csv")
print("Sleep Columns:", sleep_df.columns.tolist())


Sleep Columns: ['date', 'sleep_hours']


In [17]:
steps_df = pd.read_csv("steps.csv")
print("Steps Columns:", steps_df.columns.tolist())


Steps Columns: ['timestamp', 'steps']


In [18]:
import pandas as pd

# Load datasets
heart_df = pd.read_csv("heart_rate.csv")
sleep_df = pd.read_csv("sleep.csv")
steps_df = pd.read_csv("steps.csv")

# Rename timestamp -> date
heart_df.rename(columns={"timestamp": "date"}, inplace=True)
steps_df.rename(columns={"timestamp": "date"}, inplace=True)

# Convert date columns to datetime
heart_df['date'] = pd.to_datetime(heart_df['date'])
sleep_df['date'] = pd.to_datetime(sleep_df['date'])
steps_df['date'] = pd.to_datetime(steps_df['date'])

# Check columns
print("Heart:", heart_df.columns)
print("Sleep:", sleep_df.columns)
print("Steps:", steps_df.columns)


Heart: Index(['date', 'heart_rate'], dtype='object')
Sleep: Index(['date', 'sleep_hours'], dtype='object')
Steps: Index(['date', 'steps'], dtype='object')


In [19]:
final_df = heart_df.merge(sleep_df, on="date").merge(steps_df, on="date")
final_df.head()


,date,heart_rate,sleep_hours,steps
0,2024-01-01,72.5,6.83,22
1,2024-01-02,65.0,6.55,22
2,2024-01-03,80.9,7.70,25
3,2024-01-04,62.9,7.96,21
4,2024-01-05,69.3,7.09,24


In [20]:
final_df.to_csv("combined_fitness_data.csv", index=False)


In [32]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans

st.set_page_config(page_title="FitPulse Dashboard", layout="wide")
st.title("🏥 FitPulse Health Anomaly Detection Dashboard")

heart_file = st.file_uploader("Upload Heart Rate Dataset", type="csv")
sleep_file = st.file_uploader("Upload Sleep Dataset", type="csv")
steps_file = st.file_uploader("Upload Steps Dataset", type="csv")

if heart_file and sleep_file and steps_file:

    heart_df = pd.read_csv(heart_file)
    sleep_df = pd.read_csv(sleep_file)
    steps_df = pd.read_csv(steps_file)

    # CLEAN COLUMN NAMES
    heart_df.columns = heart_df.columns.str.strip().str.lower()
    sleep_df.columns = sleep_df.columns.str.strip().str.lower()
    steps_df.columns = steps_df.columns.str.strip().str.lower()

    # RENAME timestamp → date
    if "timestamp" in heart_df.columns:
        heart_df = heart_df.rename(columns={"timestamp": "date"})

    if "timestamp" in steps_df.columns:
        steps_df = steps_df.rename(columns={"timestamp": "date"})

    # SAFETY CHECK
    if "date" not in heart_df.columns:
        st.error("Heart Rate file missing timestamp/date column")
        st.stop()

    if "date" not in sleep_df.columns:
        st.error("Sleep file missing date column")
        st.stop()

    if "date" not in steps_df.columns:
        st.error("Steps file missing timestamp/date column")
        st.stop()

    # CONVERT TO DATETIME
    heart_df["date"] = pd.to_datetime(heart_df["date"])
    sleep_df["date"] = pd.to_datetime(sleep_df["date"])
    steps_df["date"] = pd.to_datetime(steps_df["date"])

    # MERGE
    df = heart_df.merge(sleep_df, on="date").merge(steps_df, on="date")

    st.success("✅ Data merged successfully")
    st.dataframe(df.head())

    metric = st.selectbox("Select Metric", ["heart_rate", "sleep_hours", "steps"])

    X = df[[metric]]
    kmeans = KMeans(n_clusters=2, random_state=42)
    df["cluster"] = kmeans.fit_predict(X)

    anomaly_cluster = df.groupby("cluster")[metric].mean().idxmax()
    df["anomaly"] = df["cluster"] == anomaly_cluster

    fig = px.line(df, x="date", y=metric, color="anomaly", markers=True)
    st.plotly_chart(fig, use_container_width=True)

    anomalies = df[df["anomaly"]]
    st.subheader("🚨 Detected Anomalies")
    st.dataframe(anomalies)

    csv = anomalies.to_csv(index=False).encode("utf-8")
    st.download_button("⬇ Download Report", csv, "anomaly_report.csv")


Overwriting app.py


In [33]:
from pyngrok import ngrok

ngrok.set_auth_token("389yRnYoT4XZZjShjoeHCcyhMJV_7FzMSCEysw9rKZh5JptbK")


In [34]:
!fuser -k 8501/tcp
from pyngrok import ngrok
import os

ngrok.kill()
public_url = ngrok.connect(8501)
print("🔗 Dashboard URL:", public_url)

os.system("streamlit run app.py &")


8501/tcp:             8064
🔗 Dashboard URL: NgrokTunnel: "https://compunctiously-homemade-aura.ngrok-free.dev" -> "http://localhost:8501"


0